In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
hpi = pd.DataFrame(pd.read_csv('hpi_final.csv'))
cpb = pd.DataFrame(pd.read_csv('cpb_final.csv'))
bps = pd.DataFrame(pd.read_csv('building_permits_final.csv'))
pop = pd.DataFrame(pd.read_csv('Population_Change.csv'))
inc = pd.DataFrame(pd.read_csv('county_income.csv'))

In [ ]:
pop = pop[['pop_chg_yoy','Id2', 'year','one_year', 'two_years', 'three_years', 'four_years', 'five_years']]

pop.one_year = pop.one_year.shift(-1)
pop.two_years = pop.two_years.shift(-2)
pop.three_years = pop.three_years.shift(-3)
pop.four_years = pop.four_years.shift(-4)
pop.five_years = pop.five_years.shift(-5)
pop = pop.rename(columns = {
    'Id2': 'county_code',
    'one_year': 'one_year_pop',
    'two_years': 'two_years_pop',
    'three_years': 'three_years_pop',
    'four_years': 'four_years_pop',
    'five_years': 'five_years_pop'
})

In [ ]:
inc = inc[['county_code', 'year', 'hh_income']]

gb = inc.groupby(['county_code', 'year']).mean()


def percent_change(gb):
    gb['one_year_inc']=gb['hh_income'].pct_change(periods=1).shift(-1)
    gb['two_years_inc']=gb['hh_income'].pct_change(periods=2).shift(-2)
    gb['three_years_inc']=gb['hh_income'].pct_change(periods=3).shift(-3)
    gb['four_years_inc']=gb['hh_income'].pct_change(periods=4).shift(-4)
    gb['five_years_inc']=gb['hh_income'].pct_change(periods=5).shift(-5)
    
    return gb

inc = gb.groupby(level=0).apply(percent_change)

inc.head()

In [ ]:
hpi = hpi[['county', 'year', 'hpi','one_year', 'two_years', 'three_years',
       'four_years', 'five_years'
          ]]
bps = bps[['date', 'county_code', 'single_units', 'total_units']]
cpb = cpb[['county_code', 'year', 'EMP', 'ESTAB', 'PAYANN',
           'one_year_emp',
       'two_years_emp', 'three_years_emp', 'four_years_emp', 'five_years_emp',
       'one_year_estab', 'two_years_estab', 'three_years_estab',
       'four_years_estab', 'five_years_estab', 'one_year_pay', 'two_years_pay',
       'three_years_pay', 'four_years_pay', 'five_years_pay'
          ]]
hpi.rename(columns={'county': 'county_code'}, inplace=True)
bps.rename(columns={'date': 'year'}, inplace=True)

In [ ]:
df = pd.merge(hpi, cpb, how='inner', on=['county_code', 'year'])

df1 = pd.merge(df, bps, how='inner', on=['county_code', 'year'])

df2 = pd.merge(df1, pop, how='inner', on = ['county_code', 'year'])

df3 = pd.merge(df2, inc.reset_index(), how="inner", on = ['county_code', 'year'])

In [ ]:
df3 = df3.replace([np.inf, -np.inf], np.nan)
df3 = df3.dropna()
df3.head(100)

In [ ]:
sns.set()
cols = ['five_years','five_years_pop', 'five_years_inc']
sns.pairplot(df3[cols], size = 2.5)
plt.show();

In [ ]:
g = sns.pairplot(df3, 
             y_vars=['five_years_pop', 'five_years_inc', 'five_years'],
             x_vars=['hpi', 'EMP', 'ESTAB', 'hh_income'])
g.map_lower(sns.kdeplot)
g.map_upper(sns.kdeplot)

In [ ]:
g1 = sns.pairplot(df3, vars=['hpi','one_year', 'two_years','three_years', 'four_years', 'five_years'], 
                 diag_kind='kde')
g1.map_lower(sns.kdeplot)
g1.map_upper(sns.kdeplot)

In [ ]:
g2 = sns.pairplot(df3, vars=['hh_income','one_year_inc', 'two_years_inc','three_years_inc', 'four_years_inc', 'five_years_inc'], 
                 diag_kind='kde')
g2.map_lower(sns.kdeplot)
g2.map_upper(sns.kdeplot)

In [ ]:
g3 = sns.pairplot(df3, vars=['pop_chg_yoy','one_year_pop', 'two_years_pop','three_years_pop', 'four_years_pop', 'five_years_pop'], 
                 diag_kind='kde')
g3.map_lower(sns.kdeplot)
g3.map_upper(sns.kdeplot)

In [ ]:
g4 = sns.pairplot(df3, vars=['EMP','one_year_pop', 'two_years_pop','three_years_pop', 'four_years_pop', 'five_years_pop'], 
                 diag_kind='kde')
g4.map_lower(sns.kdeplot)
g4.map_upper(sns.kdeplot)

In [ ]:
g5 = sns.pairplot(df3, 
                  x_vars=['one_year', 'two_years','three_years', 'four_years', 'five_years'],
                  y_vars=['one_year_pop', 'two_years_pop','three_years_pop', 'four_years_pop', 'five_years_pop'], 
                 )
g5.map_lower(sns.kdeplot)
g5.map_upper(sns.kdeplot)



# Machine Learning: Linear Regression

In [ ]:
X = df3[[
    'EMP', 'ESTAB', 'PAYANN', 'single_units', 'total_units',
    'one_year_emp', 'two_years_emp', 'three_years_emp', 'four_years_emp', 'five_years_emp',
       'one_year_estab', 'two_years_estab', 'three_years_estab',
       'four_years_estab', 'five_years_estab', 'one_year_pay', 'two_years_pay',
       'three_years_pay', 'four_years_pay', 'five_years_pay',
         'one_year_pop', 'one_year_inc', 'two_years_inc',
    'three_years_inc','four_years_inc','five_years_inc',
        'two_years_pop', 'three_years_pop', 'four_years_pop',
    'five_years_pop', 'hh_income']].values

l = [1.14383044e+01 5.14469879e+00 3.81888392e+00 2.50210240e+00
 1.82401920e+00 1.28582056e+00 9.11751486e-01 6.39934740e-01
 5.87864763e-01 4.69710199e-01 4.35471351e-01 3.46161050e-01
 2.63954143e-01 2.04374344e-01 1.99134938e-01 1.86972212e-01
 1.54114624e-01 1.00453651e-01 9.14101373e-02 7.14516724e-02
 6.29874665e-02 5.26596066e-02 5.01520236e-02 4.87752355e-02
 3.42196138e-02 3.16975608e-02 2.16271517e-02 1.27987425e-02
 1.15001740e-02 4.56579480e-03 5.24164182e-04]

y = df3[['five_years']].values

print(X.shape, y.shape)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data

### BEGIN SOLUTION
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train)

In [ ]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
y_train_scaled = y_scaler.transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train_scaled, y_train_scaled)

In [ ]:
predictions = model.predict(X_test_scaled)
model.fit(X_train_scaled, y_train_scaled)

In [ ]:
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = model.score(X_test_scaled, y_test_scaled)
print(f"MSE: {MSE}, R2: {r2}")


In [ ]:
from sklearn.linear_model import Lasso

### BEGIN SOLUTION
lasso = Lasso(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = lasso.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = lasso.score(X_test_scaled, y_test_scaled)
### END SOLUTION

print(f"MSE: {MSE}, R2: {r2}")

In [ ]:
from sklearn.linear_model import Ridge

### BEGIN SOLUTION
ridge = Ridge(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = ridge.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = ridge.score(X_test_scaled, y_test_scaled)
### END SOLUTION

print(f"MSE: {MSE}, R2: {r2}")

In [ ]:
from sklearn.linear_model import ElasticNet

### BEGIN SOLUTION
elasticnet = ElasticNet(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = elasticnet.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = elasticnet.score(X_test_scaled, y_test_scaled)
### END SOLUTION

print(f"MSE: {MSE}, R2: {r2}")

In [ ]:
from sklearn import linear_model
from sklearn import svm

classifiers = [
    svm.SVR(),
#     linear_model.SGDRegressor(),
    linear_model.BayesianRidge(),
#     linear_model.LassoLars(),
    linear_model.ARDRegression(),
#     linear_model.PassiveAggressiveRegressor(),
#     linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]

for item in classifiers:
    print(item)
    clf = item
    clf.fit(X_train_scaled, y_train_scaled)
    r2 = clf.score(X_test_scaled, y_test_scaled)
    print(f'r2 score: {r2}')
    print('----------------------------------------------------------------------')

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=31)
pca.fit(X_train_scaled)
print(pca.)
print('-----------------------------------------')
print(pca.explained_variance_)

In [ ]:
pca = PCA(n_components=5)
pca.fit(X_train_scaled)
X_pca = pca.transform(X_train_scaled)
print("original shape:   ", X_train_scaled.shape)
print("transformed shape:", X_pca.shape)

In [ ]:
X_new = pca.inverse_transform(X_pca)
plt.scatter(X_train_scaled[:, 0], X_train_scaled[:, 1], alpha=0.2)
plt.scatter(X_new[:, 0],X_new[:, 1], alpha=0.8)
plt.axis('equal');

In [ ]:
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=31)
Y_sklearn = sklearn_pca.fit_transform(X_train_scaled)

In [ ]:
pca = sklearn_pca.fit(X_train_scaled)
print("")
print('-----------------------------------------')
print(pca.explained_variance_)

# Deep Learning

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from keras.layers import Dense, Dropout, Flatten

In [ ]:
X = df3[[
    'EMP', 'ESTAB', 'PAYANN', 'single_units', 'total_units','hh_income','pop_chg_yoy',
    'one_year_pop', 'two_years_pop','three_years_pop', 'four_years_pop', 'five_years_pop',
    'one_year_emp', 'two_years_emp', 'three_years_emp', 'four_years_emp', 'five_years_emp',
       'one_year_estab', 'two_years_estab', 'three_years_estab',
       'four_years_estab', 'five_years_estab', 'one_year_pay', 'two_years_pay',
       'three_years_pay', 'four_years_pay', 'five_years_pay',
         'one_year_pop', 'one_year_inc', 'two_years_inc',
    'three_years_inc','four_years_inc','five_years_inc',
        'two_years_pop', 'three_years_pop', 'four_years_pop',
    'five_years_pop', 'hh_income']].values

y = df3[['five_years']].values

print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


In [ ]:
deep_model = Sequential()

In [ ]:
deep_model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))

In [ ]:
deep_model.add(Dense(100, activation='relu'))

In [ ]:
deep_model.add(Dense(y_train.shape[1], activation='softmax'))

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

# Create a StandardScater model and fit it to the training data
X_scaler = StandardScaler().fit(X_train)

In [ ]:
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from keras.utils import to_categorical

# One-hot encoding
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [ ]:
y_train_categorical

In [ ]:
# first, create a normal neural network with 2 inputs, 6 hidden nodes, and 2 outputs
from keras.models import Sequential
from keras.layers import Dense

deep_model = Sequential()
# deep_model.add(Dropout(rate=0.2, input_shape=(38,)))
deep_model.add(Dense(units=6, activation='relu', input_dim=38))
deep_model.add(Dense(units=6, activation='relu'))
deep_model.add(Dense(units=1, activation='linear'))

In [ ]:
deep_model.summary()

In [ ]:
deep_model.compile(optimizer='adam',
                  loss='mean_absolute_error',
                   metrics=['mae'])

In [ ]:
deep_model.fit(
    X_train_scaled,
    y_train,
    validation_split=0.33, 
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
model_loss, model_accuracy = deep_model.evaluate(X_test_scaled, y_test, verbose=2)

In [ ]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
y_test

In [ ]:
deep_model.predict(X_test_scaled)

In [ ]:
len(X_train)

In [ ]:
plt.scatter(deep_model.predict(X_test_scaled), y_test)

In [ ]:
plt.scatter(deep_model.predict(X_train_scaled), y_train)

In [ ]:
y_train

In [ ]:
sklearn.metrics.r2_score(y_train, deep_model.predict(X_train_scaled))

In [ ]:
sklearn.metrics.r2_score(y_test, deep_model.predict(X_test_scaled))

In [ ]:
len(y_train)

In [ ]:
len(y_test)